# Model: Xception Pretrained Keras

* [Keras Docuentaion: Applications](https://keras.io/applications/)
* [Xception model](https://keras.io/applications/#xception)
* Note: running this on a laptop (without GPU) is not feasible.

In [1]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Input, Reshape, Concatenate, GlobalAveragePooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from helper import read_mnist_data

print(tf.__version__)
print(keras.__version__)

/Users/lywang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0
2.2.2


Using TensorFlow backend.


In [2]:
DATA_PATH = '/Users/lywang/Data/mnist'
image_train, label_train, image_test, label_test = read_mnist_data(DATA_PATH)
print(image_train.shape, label_train.shape )
print(image_test.shape, label_test.shape )

# one-hot encode labels
label_train_onehot = to_categorical(label_train)
label_test_onehot = to_categorical(label_test)
print(label_train_onehot.shape )
print(label_test_onehot.shape )

(60000, 784) (60000, 1)
(10000, 784) (10000, 1)
(60000, 10)
(10000, 10)


In [3]:
MODEL_DIR = 'model_checkpoints_xception/'
model_path = os.path.join(MODEL_DIR, 'keras.{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5')

In [4]:
# restart training everytime
K.clear_session()



inputs = Input(shape=(784,))
x = Reshape((28, 28, 1))(inputs)
x = Concatenate(axis=3)([x, x, x])
base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(32, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

callback_list = [EarlyStopping(monitor='val_loss', patience=5),
                 ModelCheckpoint(model_path, monitor='val_loss', 
                                 verbose=1, save_best_only=True, 
                                 period=1)
                ]

model.fit(image_train, label_train_onehot, 
          batch_size=500, 
          epochs=100, 
          shuffle=True,
          validation_split=0.25,
          callbacks=callback_list)  # starts training


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 28, 28, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 28, 28, 3)    0           reshape_1[0][0]                  
                                                                 reshape_1[0][0]                  
                                                                 reshape_1[0][0]                  
__________________________________________________________________________________________________
block1_con

Train on 45000 samples, validate on 15000 samples
Epoch 1/100
 9850/45000 [=====>........................] - ETA: 1:48:08 - loss: 2.2399

KeyboardInterrupt: 